## Task 1
Caliberate camera using checkerboard pattern to extract camera matrix K, distortion coefficients distCoeffs and rotation and translation vectors rvec, tvec.

In [73]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
from glob import glob

def helper(
    X, Y, Z, x, y
):  # helper function to stack and get C matrix in explanation.png
    row1 = np.array([X, Y, Z, 1, 0, 0, 0, 0, -x * X, -x * Y, -x * Z, -x])
    row2 = np.array([0, 0, 0, 0, X, Y, Z, 1, -y * X, -y * Y, -y * Z, -y])
    return np.vstack((row1, row2))


def myRQ(matrix):  # QR decomposition
    Q, L = np.linalg.qr(np.linalg.inv(matrix))
    K, R = np.linalg.inv(L), Q.T
    K = K / K[2, 2]
    return K, R


# This function assumes you know the maths behind camera calibration
def get_KRP(img_coord, world_coord):
    x = img_coord[:, 0]  # extract x coordinates from image
    y = img_coord[:, 1]  # extract y coordinates from image

    X = world_coord[:, 0]  # extract X coordinates from real world cordiantes
    Y = world_coord[:, 1]  # extract Y coordinates from real world cordiantes
    Z = world_coord[:, 2]  # extract Z coordinates from real world cordiantes

    matrix = helper(
        X[0], Y[0], Z[0], x[0], y[0]
    )  # stacking the rows of the matrix to create C matrix in explanation.png
    for i in range(1, len(X)):
        to_append = helper(X[i], Y[i], Z[i], x[i], y[i])
        matrix = np.append(
            matrix, to_append, axis=0
        )  # matrix variable corresponds to C matrix in explanation.png

    u, s, vh = np.linalg.svd(matrix)  # SVD of C matrix
    P = vh[-1].reshape(
        3, 4
    )  # taking the last row of V^T and reshaping it to 3x4 matrix gives the Projection matrix P

    XYZ = np.vstack((X, Y))
    XYZ = np.vstack((XYZ, Z))
    XYZ = np.vstack(
        (XYZ, np.ones(X.shape))
    )  # Stacking the X, Y, Z, 1 to create homogenous coordinates

    xyz = P.dot(
        XYZ
    )  # multiplying P with homogenous coordinates to get the image coordinates for testing purposes to see if they are close to the original image coordinates
    xy_pred = (xyz / (xyz[2, :]))[
        0:2, :
    ]  # dividing by the last row to get the image coordinates
    xy = np.vstack(
        (x, y)
    )  # stacking the original image coordinates to compare with the predicted image coordinates

    distance = np.linalg.norm(
        xy - xy_pred
    )  # calculating the distance between the predicted image coordinates and the original image coordinates to get error

    K, R = myRQ(
        P[0:3, 0:3]
    )  # using myRQ function to decompose P into K and R matrices using QR decomposition

    return K, R, P

def frame_to_video(folder, output, fps=30):
    img = cv2.imread(os.path.join(folder, "frame1.jpg"))
    total_frames = len(os.listdir(folder))
    height, width, layers = img.shape
    size = (width, height)
    out = cv2.VideoWriter(output, cv2.VideoWriter_fourcc(*"mp4v"), fps, size)
    for x in range(total_frames):
        img = cv2.imread(os.path.join(folder, "frame{}.jpg".format(x)))
        out.write(img)
    out.release()

def video_to_frames(video, folder):
    vidcap = cv2.VideoCapture(video)
    success, image = vidcap.read()
    count = 0
    while success:
        # rotate image by 180 degrees
        image = cv2.rotate(image, cv2.ROTATE_180)
        cv2.imwrite(os.path.join(folder, "frame%d.jpg") % count, image)
        success, image = vidcap.read()
        count += 1

In [63]:
CHECKERBOARD = (7,10)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

real_world_cordinate = [(i%7,i//7,0) for i in range(42)] + [(i%7, 5, i//7 - 5) for i in range(42, 70, 1)]
real_world_cordinate = np.array(real_world_cordinate, dtype=np.float32).reshape(70,3)

imgs = []
for x in range(1, 9):
    img = cv2.imread(f"photos/perspective{x}.jpg")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
    corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria).reshape(70,2)
    K, R, P = get_KRP(corners2, real_world_cordinate)
    print("Perspective {}, k1 = {}, k2 = {}, k3 = {}, p1 = {}, p2 = {}".format(x, K[0,0], K[1,1], K[2,2], K[0,2], K[1,2]))

Perspective 1, k1 = -1986.7130780262773, k2 = -1946.7711057068045, k3 = 1.0, p1 = 524.1785665102414, p2 = 1028.6017921451937
Perspective 2, k1 = -1986.4402792210503, k2 = -1946.0184292206561, k3 = 1.0, p1 = 523.2739163226875, p2 = 1027.7845284665448
Perspective 3, k1 = -1989.706154836041, k2 = -1948.8908070121145, k3 = 1.0, p1 = 523.4412929663148, p2 = 1027.1215484315258
Perspective 4, k1 = -1987.5343802109105, k2 = -1946.6356454374145, k3 = 1.0, p1 = 524.4883569187265, p2 = 1024.3532236198255
Perspective 5, k1 = -1985.3412146427843, k2 = -1944.8604526294957, k3 = 1.0, p1 = 529.059047368865, p2 = 1024.6974532974934
Perspective 6, k1 = -1986.0742057968207, k2 = -1944.5007003450214, k3 = 1.0, p1 = 527.2492176631737, p2 = 1027.8870348548087
Perspective 7, k1 = -1988.9747772667586, k2 = -1947.3037591985703, k3 = 1.0, p1 = 531.9321387732732, p2 = 1028.8289421761306
Perspective 8, k1 = -1989.691695423917, k2 = -1948.3221310442755, k3 = 1.0, p1 = 533.6667676219189, p2 = 1032.9738338310585


## Task 2
Project a cube to the image and create the augmented reality video.

In [74]:
# convert video to frames using ffmpeg
!mkdir frames
video_to_frames("video.mp4", "frames")

mkdir: frames: File exists


In [84]:
# iterate over xyz and draw circle on image
!mkdir output
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

XYZ = np.array([[0,0,0],[0,1,0],[1,1,0],[1,0,0],[0,0,1],[1,0,1],[0,1,1],[1,1,1]], np.float32)
ones = np.ones((XYZ.shape[0],XYZ.shape[1]+1), np.float32)
ones[:,:-1] = XYZ*5
XYZ = ones
# print(XYZ.shape)

for ind, path in enumerate(glob("frames/frame*.jpg")):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    retval, corners = cv2.findChessboardCorners(gray, CHECKERBOARD)
    if retval:
        
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        _, _, P = get_KRP(corners2.reshape(70,2), real_world_cordinate)
        
        xyz = XYZ @ P.T
        xyz = xyz / xyz[:,2:]
        
        # connect the edges of the cube
        cv2.line(img, (int(xyz[0,0]),int(xyz[0,1])), (int(xyz[1,0]),int(xyz[1,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[1,0]),int(xyz[1,1])), (int(xyz[2,0]),int(xyz[2,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[2,0]),int(xyz[2,1])), (int(xyz[3,0]),int(xyz[3,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[3,0]),int(xyz[3,1])), (int(xyz[0,0]),int(xyz[0,1])), (0,255,255), 3)

        cv2.line(img, (int(xyz[4,0]),int(xyz[4,1])), (int(xyz[5,0]),int(xyz[5,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[5,0]),int(xyz[5,1])), (int(xyz[7,0]),int(xyz[7,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[6,0]),int(xyz[6,1])), (int(xyz[7,0]),int(xyz[7,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[6,0]),int(xyz[6,1])), (int(xyz[4,0]),int(xyz[4,1])), (0,255,255), 3)

        cv2.line(img, (int(xyz[0,0]),int(xyz[0,1])), (int(xyz[4,0]),int(xyz[4,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[1,0]),int(xyz[1,1])), (int(xyz[6,0]),int(xyz[6,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[2,0]),int(xyz[2,1])), (int(xyz[7,0]),int(xyz[7,1])), (0,255,255), 3)
        cv2.line(img, (int(xyz[3,0]),int(xyz[3,1])), (int(xyz[5,0]),int(xyz[5,1])), (0,255,255), 3)

        # write to output
        # convert image to BGR
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite("output/{}".format(path.split("/")[-1]), img)
    else:
        print("No corners found in {}".format(path))

mkdir: output: File exists


In [85]:
# convert frames in output to video
frame_to_video("output", "augmented_cube.mp4", 60)

In [86]:
# remove output and frames dir that were temporarily created
!rm -rf output frames